In [1]:
import os
import os.path as osp
import pandas as pd
import numpy as np
import glob

import numpy as np
import sklearn.metrics as skm

# Generate command to generate predictions

In [2]:
data_dir = '.'
conf_f = os.path.join(data_dir, 'f_channel_1.csv')  # old: f_channel.csv
conf_c = os.path.join(data_dir, 'c_channel_1.csv')  # old: c_channel.csv

In [3]:
conf_f_df = pd.read_csv(conf_f)
conf_f_df = conf_f_df.iloc[:,1:]
conf_f_df

,source_dataset,source_channel,transfer_dataset,transfer_channel,epoch
0,mass_ss1,EEG F3,mass_ss1,EEG F4,30
1,mass_ss1,EEG F4,mass_ss3,EEG F4,30
2,mass_ss1,EEG F4,isruc1,EEG F4 A1,30
3,mass_ss1,EEG F4,SleepEDF,EEG Fpz Cz,30
4,mass_ss1,EEG C4,mass_ss1,EEG F4,30
...,...,...,...,...,...
74,shhs1_normal,EEG C4 A1,SleepEDF,EEG Fpz Cz,30
75,shhs1_osa,EEG C4 A1,mass_ss1,EEG F4,30
76,shhs1_osa,EEG C4 A1,mass_ss3,EEG F4,30
77,shhs1_osa,EEG C4 A1,isruc1,EEG F4 A1,30


In [4]:
conf_c_df = pd.read_csv(conf_c)
conf_c_df = conf_c_df.iloc[:,1:]
conf_c_df

,source_dataset,source_channel,transfer_dataset,transfer_channel,epoch
0,mass_ss1,EEG F4,mass_ss1,EEG C4,30
1,mass_ss1,EEG F4,mass_ss3,EEG C4,30
2,mass_ss1,EEG F4,isruc1,EEG C4 A1,30
3,mass_ss1,EEG F4,shhs1_normal,EEG C4 A1,30
4,mass_ss1,EEG F4,shhs1_osa,EEG C4 A1,30
...,...,...,...,...,...
100,shhs1_osa,EEG C4 A1,mass_ss1,EEG C4,30
101,shhs1_osa,EEG C4 A1,mass_ss3,EEG C4,30
102,shhs1_osa,EEG C4 A1,isruc1,EEG C4 A1,30
103,shhs1_osa,EEG C4 A1,shhs1_normal,EEG C4 A1,30


In [5]:
local_data_dir = './processed'
map_db_dir = {
    'mass_ss1': 'mass_ss1',
    'mass_ss2': 'mass_ss2',
    'mass_ss3': 'mass_ss3',
    'mass_ss4': 'mass_ss4',
    'mass_ss5': 'mass_ss5',
    'SleepEDF': 'sedf_sc',
    'isruc1': 'isruc_sg1', 
    'shhs1_normal': 'shhs1_nor',
    'shhs1_osa': 'shhs1_osa',
}

In [6]:
base_cmd = 'cmd'
commands = []
count = 0
for conf_df in [conf_f_df, conf_c_df]:
    base_df = conf_df.groupby(by=['source_dataset','source_channel']).size().reset_index().rename(columns={0:'count'})
    for _, b_row in base_df.iterrows():
        # Base dataset
        b_db = str(b_row['source_dataset']).lower()
        b_ch = str(b_row['source_channel']).replace('EEG ','').replace(' ','-')
        b_dir = os.path.join(local_data_dir, map_db_dir[b_row['source_dataset']], 'views/fixed_split')

        # Base dataset
        each_base_df = conf_df[(conf_df['source_dataset'] == b_row['source_dataset']) & (conf_df['source_channel'] == b_row['source_channel'])]
        tran_df = each_base_df.groupby(by=['transfer_dataset']).size().reset_index().rename(columns={0:'count'})
        
        # Initialize command
        src_proj = f"{b_db.lower().replace(' ','_')}_{b_ch.lower()}_project"
        
        for _, t_row in tran_df.iterrows():
            t_chs = each_base_df[each_base_df['transfer_dataset'] == t_row['transfer_dataset']]['transfer_channel'].values
            count += len(t_chs)
            t_chs = '|'.join(t_chs)
            t_chs = t_chs.replace('EEG ','').replace(' ', '-')
            t_db = str(t_row['transfer_dataset']).lower()
            t_dir = os.path.join(local_data_dir, map_db_dir[t_row['transfer_dataset']], 'views/fixed_split')
            tgt_proj = f"{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project"
            
            check1 = osp.isdir(f"tf_{src_proj}__to__{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project")
            check2 = osp.isfile(f"{src_proj}/model/model_weights.h5")
            check3 = osp.isfile(f"{tgt_proj}/model/model_weights.h5")
            if check1 and check2 and check3:
                print(f"cd tf_{src_proj}__to__{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project")
                print(f"ut predict --one_shot --save_true --out_dir ../predictions/{src_proj}__to__{tgt_proj}_type3 --weights_file_name '../{src_proj}/model/model_weights.h5' --overwrite")
                print(f"ut predict --one_shot --save_true --out_dir ../predictions/{src_proj}__to__{tgt_proj}_type6 --weights_file_name '../{tgt_proj}/model/model_weights.h5' --overwrite")
                print(f"ut predict --one_shot --save_true --out_dir ../predictions/{src_proj}__to__{tgt_proj}_type9 --overwrite")
                print("cd ..")
            # else:
            #     print(f" ** Skip tf_{src_proj}__to__{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project")

cd tf_sleepedf_fpz-cz_project__to__isruc1_f4-a1_project
ut predict --one_shot --save_true --out_dir ../predictions/sleepedf_fpz-cz_project__to__isruc1_f4-a1_project_type3 --weights_file_name '../sleepedf_fpz-cz_project/model/model_weights.h5' --overwrite
ut predict --one_shot --save_true --out_dir ../predictions/sleepedf_fpz-cz_project__to__isruc1_f4-a1_project_type6 --weights_file_name '../isruc1_f4-a1_project/model/model_weights.h5' --overwrite
ut predict --one_shot --save_true --out_dir ../predictions/sleepedf_fpz-cz_project__to__isruc1_f4-a1_project_type9 --overwrite
cd ..
cd tf_sleepedf_fpz-cz_project__to__mass_ss1_f4_project
ut predict --one_shot --save_true --out_dir ../predictions/sleepedf_fpz-cz_project__to__mass_ss1_f4_project_type3 --weights_file_name '../sleepedf_fpz-cz_project/model/model_weights.h5' --overwrite
ut predict --one_shot --save_true --out_dir ../predictions/sleepedf_fpz-cz_project__to__mass_ss1_f4_project_type6 --weights_file_name '../mass_ss1_f4_project/model

# Calculate performance metrics

In [7]:
results = []
for conf_df in [conf_f_df, conf_c_df]:
    base_df = conf_df.groupby(by=['source_dataset','source_channel']).size().reset_index().rename(columns={0:'count'})
    for _, b_row in base_df.iterrows():
        # Base dataset
        b_db = str(b_row['source_dataset']).lower()
        b_ch = str(b_row['source_channel']).replace('EEG ','').replace(' ','-')
        b_dir = os.path.join(local_data_dir, map_db_dir[b_row['source_dataset']], 'views/fixed_split')

        # Base dataset
        each_base_df = conf_df[(conf_df['source_dataset'] == b_row['source_dataset']) & (conf_df['source_channel'] == b_row['source_channel'])]
        tran_df = each_base_df.groupby(by=['transfer_dataset']).size().reset_index().rename(columns={0:'count'})
        
        # Initialize command
        src_proj = f"{b_db.lower().replace(' ','_')}_{b_ch.lower()}_project"
        
        for _, t_row in tran_df.iterrows():
            t_chs = each_base_df[each_base_df['transfer_dataset'] == t_row['transfer_dataset']]['transfer_channel'].values
            count += len(t_chs)
            t_chs = '|'.join(t_chs)
            t_chs = t_chs.replace('EEG ','').replace(' ', '-')
            t_db = str(t_row['transfer_dataset']).lower()
            t_dir = os.path.join(local_data_dir, map_db_dir[t_row['transfer_dataset']], 'views/fixed_split')
            tgt_proj = f"{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project"
            
            a_result = [
                f"{b_db.lower().replace(' ','_')}", f"eeg_{b_ch.lower().replace('-','_')}",
                f"{t_db.lower().replace(' ','_')}", f"eeg_{t_chs.lower().replace('-','_')}",
            ]
            for t in [3,6,9]:
                scan_gt_path = osp.join(
                    f"predictions/{src_proj}__to__{tgt_proj}_type{t}", 'test_data', 'dataset_1', '*_TRUE.npy'
                )
                gt_files = list(glob.glob(scan_gt_path))
                if len(gt_files) > 0:
                    y_trues = []
                    y_preds = []
                    for fp_gt in gt_files:
                        fp_pred = fp_gt.replace('_TRUE.npy', '_PRED.npy')
                        y_true = np.squeeze(np.load(fp_gt))
                        y_pred = np.load(fp_pred)
                        y_trues.extend(y_true)
                        y_preds.extend(y_pred)
                    y_trues, y_preds = np.array(y_trues), np.array(y_preds)
                    a_result.extend([
                        skm.accuracy_score(y_trues, y_preds),
                        skm.f1_score(y_trues, y_preds, average='macro')
                    ])
                    # print(f"{src_proj}__to__{tgt_proj}_type{t}")
                    # print(y_trues.shape, y_preds.shape)
                    # print(f"ACC: {skm.accuracy_score(y_trues, y_preds)}")
                    # print(f"MF1: {skm.f1_score(y_trues, y_preds, average='macro')}")
                    # print(f"PR: {skm.precision_score(y_trues, y_preds, average=None, labels=[0,1,2,3,4])}")
                    # print(f"RE: {skm.recall_score(y_trues, y_preds, average=None, labels=[0,1,2,3,4])}")
                    # print(f"F1: {skm.f1_score(y_trues, y_preds, average=None, labels=[0,1,2,3,4])}")
            results.append(a_result)
            
tf_df = pd.DataFrame(results, columns=[
    'source_dataset','source_channel','target_dataset','target_channel',
    'acc_types3','mf1_type3','acc_types6','mf1_type6','acc_types9','mf1_type9'])

In [8]:
tf_df

,source_dataset,source_channel,target_dataset,target_channel,acc_types3,mf1_type3,acc_types6,mf1_type6,acc_types9,mf1_type9
0,sleepedf,eeg_fpz_cz,isruc1,eeg_f4_a1,0.623793,0.554979,0.783908,0.755599,0.776667,0.744184
1,sleepedf,eeg_fpz_cz,mass_ss1,eeg_f4,0.651190,0.581189,0.812585,0.746733,0.810544,0.738406
2,sleepedf,eeg_fpz_cz,mass_ss3,eeg_f4,0.764715,0.651121,0.863711,0.796492,0.865209,0.795972
3,sleepedf,eeg_pz_oz,sleepedf,eeg_fpz_cz,0.870257,0.674071,0.922836,0.759496,0.927810,0.778479
4,sleepedf,eeg_pz_oz,isruc1,eeg_f4_a1,0.582299,0.531925,0.784023,0.755696,0.779655,0.749826
...,...,...,...,...,...,...,...,...,...,...
179,shhs1_osa,eeg_c3_a2,shhs1_osa,eeg_c4_a1,0.804689,0.687771,0.776206,0.660408,0.827359,0.721824
180,shhs1_osa,eeg_c4_a1,isruc1,eeg_c4_a1,0.713793,0.644805,0.788621,0.756630,0.787931,0.757334
181,shhs1_osa,eeg_c4_a1,mass_ss1,eeg_c4,0.739796,0.644528,0.808333,0.735388,0.821599,0.743103
182,shhs1_osa,eeg_c4_a1,mass_ss3,eeg_c4,0.815186,0.706220,0.860416,0.787830,0.870451,0.809665


In [9]:
tf_df.to_csv('utime_conclusions.csv', index=False)